In [4]:
import numpy as np
import matplotlib.pyplot as plt
import polars as pl
import sklearn
import os, pathlib, shutil
import kaggle, kagglehub


Download the competition


In [5]:
competition_name = "home-data-for-ml-course"  # Change this to the competition you want to download
competition_path = pathlib.Path(
    kagglehub.competition.competition_download(competition_name)
)  # Downloads the competition
competition_files: list = os.listdir(competition_path)  # List of the files included in the competition

print(competition_files)

# copy files to the current directory
shutil.copyfile(competition_path / "sample_submission.csv", "sample_submission.csv")
shutil.copyfile(competition_path / "test.csv", "test.csv")
shutil.copyfile(competition_path / "train.csv", "train.csv")

['data_description.txt', 'sample_submission.csv', 'sample_submission.csv.gz', 'test.csv', 'test.csv.gz', 'train.csv', 'train.csv.gz']


'train.csv'

In [10]:
pl.Config.set_tbl_rows(50)
train_df = pl.read_csv(competition_path / "train.csv", ignore_errors=True)
test_df = pl.read_csv(competition_path / "test.csv", ignore_errors=True)
sample_submission_df = pl.read_csv(competition_path / "sample_submission.csv")

target_feature_name: str = (set(train_df.columns) - set(test_df.columns)).pop()
target_col = train_df.get_column(target_feature_name)
print(train_df.drop(target_feature_name).schema == test_df.schema)
df = pl.concat([train_df.drop(target_feature_name), test_df])

True


### EDA


In [12]:
numeric_cols_df = train_df.drop("Id").select(pl.selectors.numeric())
pearson_correlations = (
    numeric_cols_df.select(pl.corr(pl.col("*"), target_feature_name).abs())
    .transpose(include_header=True)
    .sort("column_0", descending=True)
)
print(pearson_correlations)

shape: (35, 2)
┌───────────────┬──────────┐
│ column        ┆ column_0 │
│ ---           ┆ ---      │
│ str           ┆ f64      │
╞═══════════════╪══════════╡
│ SalePrice     ┆ 1.0      │
│ OverallQual   ┆ 0.790982 │
│ GrLivArea     ┆ 0.708624 │
│ GarageCars    ┆ 0.640409 │
│ GarageArea    ┆ 0.623431 │
│ TotalBsmtSF   ┆ 0.613581 │
│ 1stFlrSF      ┆ 0.605852 │
│ FullBath      ┆ 0.560664 │
│ TotRmsAbvGrd  ┆ 0.533723 │
│ YearBuilt     ┆ 0.522897 │
│ YearRemodAdd  ┆ 0.507101 │
│ MasVnrArea    ┆ 0.477493 │
│ Fireplaces    ┆ 0.466929 │
│ BsmtFinSF1    ┆ 0.38642  │
│ WoodDeckSF    ┆ 0.324413 │
│ 2ndFlrSF      ┆ 0.319334 │
│ OpenPorchSF   ┆ 0.315856 │
│ HalfBath      ┆ 0.284108 │
│ LotArea       ┆ 0.263843 │
│ BsmtFullBath  ┆ 0.227122 │
│ BsmtUnfSF     ┆ 0.214479 │
│ BedroomAbvGr  ┆ 0.168213 │
│ KitchenAbvGr  ┆ 0.135907 │
│ EnclosedPorch ┆ 0.128578 │
│ ScreenPorch   ┆ 0.111447 │
│ PoolArea      ┆ 0.092404 │
│ MSSubClass    ┆ 0.084284 │
│ OverallCond   ┆ 0.077856 │
│ MoSold        ┆ 0.046432 │